# 1 pacotes 

In [77]:
import mwclient
import time
import datetime as dt
from datetime import date, timedelta
import pandas as pd
import json

## 2 Obter MatchId a partir da data 

In [2]:
def obterMatchId(dataJogo:str,**kwargs):
    
    liga = kwargs.get('liga','')
    dataJogoFim = kwargs.get('dataJogoFim','')
    
    date = dt.datetime.strptime(dataJogo, "%Y-%m-%d").date()
    dateFinal = dt.datetime.strptime(dataJogoFim, "%Y-%m-%d").date() if len(dataJogoFim)>0 else date
    
    cond = "M.DateTime_UTC >= '" + str(date) + " 00:00:00' AND M.DateTime_UTC <= '" + str(dateFinal+dt.timedelta(1)) + " 00:00:00'"
    if len(liga)>0:
        cond = cond+" AND "+f"M.MatchId like '%{liga}%'"
    
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = 'max',
        tables = "MatchSchedule=M",
        fields = "M.MatchId,M.DateTime_UTC,M.PageAndTeam1, M.PageAndTeam2, M.Team1,M.Team2, M.BestOf,M.Patch,M.PatchPage,M.VodHighlights",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [3]:
obterMatchId('2022-07-02', liga = 'CBLOL',dataJogoFim='2022-07-03')

,MatchId,DateTime UTC,PageAndTeam1,PageAndTeam2,Team1,Team2,BestOf,Patch,PatchPage,VodHighlights,DateTime UTC__precision
0,CBLOL/2022 Season/Split 2_Week 4_1,2022-07-02 16:00:00,CBLOL/2022 Season/Split 2_Miners,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,Miners,Flamengo Los Grandes,1,12.12,Patch 12.12,,0
1,CBLOL/2022 Season/Split 2_Week 4_10,2022-07-03 20:00:00,CBLOL/2022 Season/Split 2_Flamengo Los Grandes,CBLOL/2022 Season/Split 2_FURIA Esports,Flamengo Los Grandes,FURIA Esports,1,12.12,Patch 12.12,,0
2,CBLOL/2022 Season/Split 2_Week 4_2,2022-07-02 17:00:00,CBLOL/2022 Season/Split 2_Rensga Esports,CBLOL/2022 Season/Split 2_LOUD,Rensga Esports,LOUD,1,12.12,Patch 12.12,,0
3,CBLOL/2022 Season/Split 2_Week 4_3,2022-07-02 18:00:00,CBLOL/2022 Season/Split 2_RED Canids,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,RED Canids,KaBuM! e-Sports,1,12.12,Patch 12.12,,0
4,CBLOL/2022 Season/Split 2_Week 4_4,2022-07-02 19:00:00,CBLOL/2022 Season/Split 2_paiN Gaming,CBLOL/2022 Season/Split 2_FURIA Esports,paiN Gaming,FURIA Esports,1,12.12,Patch 12.12,,0
5,CBLOL/2022 Season/Split 2_Week 4_5,2022-07-02 20:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_Liberty,INTZ,Liberty,1,12.12,Patch 12.12,,0
6,CBLOL/2022 Season/Split 2_Week 4_6,2022-07-03 16:00:00,CBLOL/2022 Season/Split 2_KaBuM! e-Sports,CBLOL/2022 Season/Split 2_Rensga Esports,KaBuM! e-Sports,Rensga Esports,1,12.12,Patch 12.12,,0
7,CBLOL/2022 Season/Split 2_Week 4_7,2022-07-03 17:00:00,CBLOL/2022 Season/Split 2_Liberty,CBLOL/2022 Season/Split 2_RED Canids,Liberty,RED Canids,1,12.12,Patch 12.12,,0
8,CBLOL/2022 Season/Split 2_Week 4_8,2022-07-03 18:00:00,CBLOL/2022 Season/Split 2_INTZ,CBLOL/2022 Season/Split 2_paiN Gaming,INTZ,paiN Gaming,1,12.12,Patch 12.12,,0
9,CBLOL/2022 Season/Split 2_Week 4_9,2022-07-03 19:00:00,CBLOL/2022 Season/Split 2_LOUD,CBLOL/2022 Season/Split 2_Miners,LOUD,Miners,1,12.12,Patch 12.12,,0


## Obter video de gravação através do MatchId

In [4]:
def obterVideoFromGamedId(macthid):
    
    cond = f"MG.MatchId = '{macthid}'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "MatchScheduleGame=MG",
        fields = "MG.GameId,MG.MatchId,MG.Blue,MG.Red,MG.Vod,MG.VodPostgame,MG.MatchHistory,MG.N_GameInMatch",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [5]:
obterVideoFromGamedId('CBLOL/2022 Season/Split 2_Week 4_1')

,GameId,MatchId,Blue,Red,Vod,VodPostgame,MatchHistory,N GameInMatch
0,CBLOL/2022 Season/Split 2_Week 4_1_1,CBLOL/2022 Season/Split 2_Week 4_1,Miners,Flamengo Los Grandes,https://www.youtube.com/watch?v=oGwlB1RDfYQ,,,1


## Obter informações do time através do GameId

In [6]:
def obterScoreFromGameId(gameId):
    
    cond = f"SG.GameId like '%{gameId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardGames=SG",
        fields = "SG.GameId,RiotGameId,Tournament,SG.DateTime_UTC,SG.Gamelength,Gamelength_Number,Patch,PatchSort,VOD,Team1Dragons,Team1Barons,Team1Towers,Team1Gold,Team1Kills,Team1RiftHeralds,Team1Inhibitors,MatchHistory,Team2Dragons,Team2Barons,Team2Towers,Team2Gold,Team2Kills,Team2RiftHeralds,Team2Inhibitors",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [7]:
obterScoreFromGameId('CBLOL/2022 Season/Split 2_Week 4_1_1')

,GameId,RiotGameId,Tournament,DateTime UTC,Gamelength,Gamelength Number,Patch,PatchSort,VOD,Team1Dragons,...,Team1Inhibitors,MatchHistory,Team2Dragons,Team2Barons,Team2Towers,Team2Gold,Team2Kills,Team2RiftHeralds,Team2Inhibitors,DateTime UTC__precision
0,CBLOL/2022 Season/Split 2_Week 4_1_1,,CBLOL 2022 Split 2,2022-07-02 16:19:00,34:49,34.816667,12.12,,https://www.youtube.com/watch?v=oGwlB1RDfYQ,4,...,1,,1,0,1,55525,6,0,0,0


## Obter informação dos times pelo GameId

In [8]:
def obterTeamScoreFromGameId(gameId,**kwargs):
    
    cond = f"ST.GameId like '%{gameId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardTeams=ST",
        fields = "ST.GameId,ST.Team,IsWinner,Side,Number,GameTeamId,UniqueTeam",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [9]:
ts = obterTeamScoreFromGameId('CBLOL/2022 Season/Split 2_Week 4_1_1')

## Obter informação dos jogadoes pelo GameTeamId

In [10]:
ts.iloc[0,5]

'CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1_1_Team2'

In [13]:
def obterPlayerScoreFromGameTeamId(GameTeamId,**kwargs):
    
    cond = f"SP.GameTeamId like '%{GameTeamId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = '10',
        tables = "ScoreboardPlayers=SP",
        fields = "GameId,GameTeamId,Name,Role,Role_Number,IngameRole,Link,Champion,Kills,Deaths,Assists,Gold,CS,DamageToChampions,VisionScore,UniqueRole,UniqueRoleVs,OverviewPage",
        where  = cond
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [14]:
obterPlayerScoreFromGameTeamId(ts.iloc[1,5])

,GameTeamId,Name,Role,Role Number,IngameRole,Link,Champion,Kills,Deaths,Assists,Gold,CS,DamageToChampions,VisionScore,UniqueRole,UniqueRoleVs,OverviewPage
0,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,Anyyy,Mid,3,Mid,Anyyy,Azir,1,2,8,12750,291,17854,41,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2
1,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,Celo,Bot,4,Bot,Celo,Seraphine,2,1,13,15082,318,12870,44,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2
2,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,DoRun,Top,1,Top,DoRun,Gnar,3,0,11,16371,338,15681,54,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2
3,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,Hugato,Jungle,2,Jungle,Hugato,Xin Zhao,4,1,5,12754,230,7048,54,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2
4,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,Sive,Support,5,Support,Sive,Pyke,6,2,8,10904,57,7303,162,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2


## obter informações de um único jogador

In [27]:
def obterPlayerScoreFromLink(LINK,**kwargs):
    
    cond = f"SP.Link like '%{LINK}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = 'max',
        tables = "ScoreboardPlayers=SP",
        fields = "GameId,GameTeamId,Name,Role,Role_Number,IngameRole,Link,Champion,Kills,Deaths,Assists,Gold,CS,DamageToChampions,VisionScore,UniqueRole,UniqueRoleVs,OverviewPage,DateTime_UTC",
        where  = cond,
        order_by = 'DateTime_UTC'
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [28]:
pp = obterPlayerScoreFromGameTeamId(ts.iloc[1,5])

In [29]:
pp.loc[0,'Link']

'Anyyy'

In [30]:
obterPlayerScoreFromLink(pp.loc[0,'Link'])

,GameId,GameTeamId,Name,Role,Role Number,IngameRole,Link,Champion,Kills,Deaths,Assists,Gold,CS,DamageToChampions,VisionScore,UniqueRole,UniqueRoleVs,OverviewPage,DateTime UTC,DateTime UTC__precision
0,CBLOL/2018 Season/Split 2_Week 1_4_1,CBLOL/2018 Season/Split 2/Scoreboards_4_1_Team2,Anyyy,Mid,3,Mid,Anyyy,Yasuo,7,3,8,20549,431,,,CBLOL/2018 Season/Split 2/Scoreboards_4_1_2_3,CBLOL/2018 Season/Split 2/Scoreboards_4_1_1_3,CBLOL/2018 Season/Split 2,2018-06-10 18:52:00,0
1,CBLOL/2018 Season/Split 2_Week 1_4_2,CBLOL/2018 Season/Split 2/Scoreboards_4_2_Team1,Anyyy,Mid,3,Mid,Anyyy,Vladimir,0,3,0,9260,243,,,CBLOL/2018 Season/Split 2/Scoreboards_4_2_1_3,CBLOL/2018 Season/Split 2/Scoreboards_4_2_2_3,CBLOL/2018 Season/Split 2,2018-06-10 20:14:00,0
2,CBLOL/2018 Season/Split 2_Week 1_4_3,CBLOL/2018 Season/Split 2/Scoreboards_4_3_Team2,Anyyy,Mid,3,Mid,Anyyy,Irelia,6,4,13,14121,256,,,CBLOL/2018 Season/Split 2/Scoreboards_4_3_2_3,CBLOL/2018 Season/Split 2/Scoreboards_4_3_1_3,CBLOL/2018 Season/Split 2,2018-06-10 21:08:00,0
3,CBLOL/2018 Season/Split 2_Week 2_4_1,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,Anyyy,Mid,3,Mid,Anyyy,Zoe,1,1,1,10044,269,,,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,CBLOL/2018 Season/Split 2,2018-06-16 20:11:00,0
4,CBLOL/2018 Season/Split 2_Week 2_4_2,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,Anyyy,Mid,3,Mid,Anyyy,Yasuo,2,3,0,9495,249,,,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,CBLOL/2018 Season/Split 2/Scoreboards/Week 2_4...,CBLOL/2018 Season/Split 2,2018-06-16 21:12:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,CBLOL/2022 Season/Split 2_Week 3_8_1,CBLOL/2022 Season/Split 2/Scoreboards/Week 3_8...,Anyyy,Mid,3,Mid,Anyyy,Orianna,1,3,3,11752,290,14468,30,CBLOL/2022 Season/Split 2/Scoreboards/Week 3_8...,CBLOL/2022 Season/Split 2/Scoreboards/Week 3_8...,CBLOL/2022 Season/Split 2,2022-06-26 19:34:00,0
204,CBLOL/2022 Season/Split 2_Week 4_1_1,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,Anyyy,Mid,3,Mid,Anyyy,Azir,1,2,8,12750,291,17854,41,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2,2022-07-02 16:19:00,0
205,CBLOL/2022 Season/Split 2_Week 4_9_1,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_9...,Anyyy,Mid,3,Mid,Anyyy,Swain,2,2,3,9912,227,14388,28,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_9...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_9...,CBLOL/2022 Season/Split 2,2022-07-03 19:21:00,0
206,CBLOL/2022 Season/Split 2_Week 5_5_1,CBLOL/2022 Season/Split 2/Scoreboards/Week 5_5...,Anyyy,Mid,3,Mid,Anyyy,Sylas,4,2,4,12800,288,13680,23,CBLOL/2022 Season/Split 2/Scoreboards/Week 5_5...,CBLOL/2022 Season/Split 2/Scoreboards/Week 5_5...,CBLOL/2022 Season/Split 2,2022-07-09 20:37:00,0


# Json RiotGame

## 1 obter pages através do GameId

In [35]:
def obterPageJSON(GameId,**kwargs):
    
    cond = f"PJ.GameId like '%{GameId}%'"
    site = mwclient.Site('lol.fandom.com', path='/')
    response = site.api(
        'cargoquery',
        limit = 'max',
        tables = "PostgameJsonMetadata=PJ",
        fields = "GameId,MatchId,StatsPage,TimelinePage,OverviewPage",
        where  = cond,
    )
    
    jsonData = json.dumps(response['cargoquery'])
    ParseData = jsonData.replace('''{"title\":''','').replace("}}",'}')
    df = pd.read_json(ParseData, orient = 'values' )
    
    return df

In [36]:
ts

,GameId,Team,IsWinner,Side,Number,GameTeamId,UniqueTeam
0,CBLOL/2022 Season/Split 2_Week 4_1_1,Flamengo Los Grandes,0,Red,2,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...
1,CBLOL/2022 Season/Split 2_Week 4_1_1,Miners,1,Blue,1,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...


In [37]:
obterPageJSON(ts.loc[0,'GameId'])

,GameId,MatchId,StatsPage,TimelinePage,OverviewPage
0,CBLOL/2022 Season/Split 2_Week 4_1_1,CBLOL/2022 Season/Split 2_Week 4_1,V5 data:ESPORTSTMNT04 2410410,V5 data:ESPORTSTMNT04 2410410/Timeline,CBLOL/2022 Season/Split 2


## obter dict from page

In [43]:
def obterPage(pageLink):
    site = mwclient.Site('lol.fandom.com', path='/')
    page =  site.pages[pageLink]
    textAux = page.text()
    return json.loads(textAux)

In [44]:
obterPage('V5 data:ESPORTSTMNT04 2410410')

{'gameCreation': 1656776711000,
 'gameDuration': 2089,
 'gameEndTimestamp': 1656781031363,
 'gameId': 2410410,
 'gameMode': 'CLASSIC',
 'gameName': '108211865723846616|game1',
 'gameStartTimestamp': 1656778794516,
 'gameType': 'CUSTOM_GAME',
 'gameVersion': '12.12.448.6653',
 'mapId': 11,
 'participants': [{'assists': 11,
   'baronKills': 0,
   'bountyLevel': 3,
   'challenges': {'12AssistStreakCount': 0,
    'abilityUses': 266,
    'acesBefore15Minutes': 0,
    'alliedJungleMonsterKills': 10,
    'baronBuffGoldAdvantageOverThreshold': 1,
    'baronTakedowns': 1,
    'blastConeOppositeOpponentCount': 0,
    'bountyGold': 0,
    'buffsStolen': 1,
    'completeSupportQuestInTime': 0,
    'controlWardTimeCoverageInRiverOrEnemyHalf': 0.47485163207387826,
    'controlWardsPlaced': 11,
    'damagePerMinute': 450.34737327807534,
    'damageTakenOnTeamPercentage': 0.23681448632851798,
    'dancedWithRiftHerald': 0,
    'deathsByEnemyChamps': 0,
    'dodgeSkillShotsSmallWindow': 1,
    'doubleA

## obter json da partida -> lista(GameId,statusPage,timeLinePage)

In [49]:
def obterRiotGameMatch(GameId,**kwargs)->list:
    df = obterPageJSON(GameId)
    partidaInfo ={
        'GameId':GameId,
        'StatsPage':obterPage(df.loc[0,'StatsPage']),
        'TimelinePage':obterPage(df.loc[0,'TimelinePage'])
    }    
    return partidaInfo

In [50]:
ts

,GameId,Team,IsWinner,Side,Number,GameTeamId,UniqueTeam
0,CBLOL/2022 Season/Split 2_Week 4_1_1,Flamengo Los Grandes,0,Red,2,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...
1,CBLOL/2022 Season/Split 2_Week 4_1_1,Miners,1,Blue,1,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...,CBLOL/2022 Season/Split 2/Scoreboards/Week 4_1...


In [51]:
gameJSON = obterRiotGameMatch(ts.loc[0,'GameId'])

In [82]:
timestamp = float(gameJSON.get('StatsPage').get('gameStartTimestamp'))

In [88]:
A1 = dt.datetime.fromtimestamp(timestamp*0.001)

In [89]:
print(A1)

2022-07-02 13:19:54.516000


In [80]:
gameJSON.get('StatsPage').keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'seasonId', 'teams', 'tournamentCode'])

In [81]:
gameJSON.get('StatsPage').get('gameStartTimestamp')

1656778794516